In [2]:
import pandas as pd

import sys
sys.path.append('../')

import os

In [21]:
date = '2025-01-19'
data = pd.read_csv('../data/oresund/'+date+'.csv')
data = data.drop_duplicates()

In [86]:
#Getting a quick overview of the data
nrships = len(data['MMSI'].unique())
lendata = len(data)

print('Number of ships: ', nrships ,
      '\nNumber of data points: ', lendata)

print(data.columns)

print(data['Ship type'].unique())

print(data['Navigational status'].unique()) #I don't really want to filter on dynamical statuses
#because they can be false...

print(data['# Timestamp'].head())

Number of ships:  379 
Number of data points:  616606
Index(['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude',
       'Navigational status', 'SOG', 'COG', 'Heading', 'Ship type',
       'Cargo type', 'Width', 'Length', 'Destination'],
      dtype='object')
['Undefined' 'Cargo' 'Other' 'Passenger' 'Pleasure' 'Sailing'
 'Port tender' 'Pilot' 'Dredging' 'SAR' 'Tug' 'Military' 'Towing' 'Tanker'
 'Diving' 'HSC' 'Reserved' 'Fishing' 'Law enforcement']
['Under way using engine' 'Unknown value' 'Moored'
 'Restricted maneuverability' 'At anchor' 'Not under command'
 'Power-driven vessel pushing ahead or towing alongside']
0    19/01/2025 00:00:00
1    19/01/2025 00:00:00
2    19/01/2025 00:00:00
6    19/01/2025 00:00:00
7    19/01/2025 00:00:00
Name: # Timestamp, dtype: object


In [96]:
data_filter = data.copy()

#I filter for helicopter
data_filter = data_filter[(data_filter['SOG'] < 50) | (data_filter['SOG'].isna())] #A helicopter is faster than 50 knots and some ships are noobs
print(len(data_filter['MMSI'].unique()))

data_temp = pd.DataFrame(columns=data_filter.columns)
#I need to filter down the time if things are to be scalable
#This does it one a five minute basis
for MMSI in data_filter['MMSI'].unique():
    ship = data_filter[data_filter['MMSI'] == MMSI]
    #Creating a series for time filtering
    series = pd.Series(range(len(ship)), index=pd.to_datetime(ship['# Timestamp'], dayfirst=True))
    #Resampling the series to 5 minutes. This gives the index of the last data point in the 5 minute interval
    #The unique is added because if transmission occurs less frequently than 5 mins there will be duplicates
    resampled = (series.resample('5min').count().cumsum()-1).unique()
    #Filtering the data
    ship = ship.iloc[resampled]
    data_temp = pd.concat([data_temp, ship], ignore_index=True)

data_filter = data_temp.copy()


378


/var/folders/29/jqsnm5js6_sc7663vjsq3mp40000gp/T/ipykernel_40760/1930749798.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_temp = pd.concat([data_temp, ship], ignore_index=True)


In [101]:
#Saving the filtered data
date = '2025-01-19'
filename = '../data/oresund/'+date+'_filtered.csv'
if not os.path.exists(filename):
    data_filter.to_csv(filename, index=False)

down here is me fooling around to learn the relevant commands

In [34]:
#this is me fooling around with timestamps
dt = pd.to_datetime(data_filter['# Timestamp'])
print(dt)
(dt.iloc[-1] - dt.iloc[0]).total_seconds()/(60*60)

0         2025-01-19 00:00:00
1         2025-01-19 00:00:00
2         2025-01-19 00:00:00
6         2025-01-19 00:00:00
7         2025-01-19 00:00:00
                  ...        
1936779   2025-01-19 23:59:58
1936780   2025-01-19 23:59:58
1936786   2025-01-19 23:59:58
1936788   2025-01-19 23:59:58
1936793   2025-01-19 23:59:58
Name: # Timestamp, Length: 615750, dtype: datetime64[ns]


/var/folders/29/jqsnm5js6_sc7663vjsq3mp40000gp/T/ipykernel_40760/3133416223.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(data_filter['# Timestamp'])


23.999444444444446

In [84]:
#Fooling around with MMSI number
MMSI_ar = data_filter['MMSI'].unique()
ship = data_filter[data_filter['MMSI'] == MMSI_ar[0]]
ship_time = pd.to_datetime(ship['# Timestamp'])
ship_time.index = ship_time
new_ship_time = ship_time.iloc[ship_time.resample('1min').count().cumsum()-1]

print(len(ship_time)/len(new_ship_time))

#How about if it does not signal every minute?
index = pd.date_range(start=ship_time.index[0], end=ship_time.index[-1], freq='3min')
series = pd.Series(range(len(index)), index=index)
series.resample('1min').count().cumsum().unique()

#I need the unique part because the cumsum will give me the same value for all the 3 minute intervals

4.884027777777778


/var/folders/29/jqsnm5js6_sc7663vjsq3mp40000gp/T/ipykernel_40760/3016659332.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ship_time = pd.to_datetime(ship['# Timestamp'])


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

Adding new stuff 2/3 2025

In [11]:
data = pd.read_csv('../data/oresund/2025-01-19_filtered.csv')
print(data['Type of mobile'].unique(), len(data))
data = data[(data['Type of mobile']=='Class A') | (data['Type of mobile']=='Class B')]
print(data['Type of mobile'].unique(), len(data))

['Class A' 'Class B' 'Base Station' 'AtoN'] 69149
['Class A' 'Class B'] 67423


In [13]:
#Saving data 2/3 2025
date = '2025-01-19'
filename = '../data/oresund/'+date+'_filtered_2.csv'
if not os.path.exists(filename):
    data.to_csv(filename, index=False)